### Instructions
A - above
B - below
X - delete 
Ctrl Enter - run
Enter - Enter into cell
M - make cell into markdown cell

In [60]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot

In [61]:
f = open("modules.txt", 'r')
#print(f.read())
f.close()

In [65]:
global allSplits
allSplits = []
f = open("modules.txt", 'r')
lineCount = 0
for line in f:
    allSplits.append(line.split("|"))
    lineCount+=1
f.close()

global modSplits
modSplits = []
global modConstraintSplits
modConstraintSplits = []
for line in allSplits:
    modSplits.append(line[0])
    modConstraintSplits.append(line[3])

In [ ]:
#Part1


# THIS IS A HEADING. ITS NEAT.
# BELOW IT IS SOME CODE.

In [66]:
def generate_timetable():
    """
    Generate a timetable Spyx
    Read the file to see how many classes + labs there are.
    Store them in a 20x3 list alongside empty sessions to fill the list
    """
    #Create "empty" timetable
    timetable = [["empty" for x in range(3)] for y in range(20)] 
    
    #Read file into allSplits list
    """
    allSplits = []
    f = open("modules.txt", 'r')
    lineCount = 0
    for line in f:
        allSplits.append(line.split("|"))
        lineCount+=1
    f.close()
    """
    
    #Add module into timetable
    for i in range(0,lineCount):
        timetable[i][0] = allSplits[i][0]
    
    
    #Add seminars into timetable.
    #Iterate through each module, for each number of seminars use
    #counters to iterate to next empty position and place seminar.
    #Should work on different set of modules if format is kept identical.
    emptyChecker = 0
    emptyChecker1 = 0
    for i in range(0, lineCount):
        for j in range(0, int(allSplits[i][2])):
            toggle = False
            while toggle == False:
                if timetable[emptyChecker][emptyChecker1] == "empty":
                    timetable[emptyChecker][emptyChecker1] = allSplits[i][0]
                    toggle = True
                emptyChecker1+=1
                if emptyChecker1 == 3:
                    emptyChecker+=1
                    emptyChecker1=0
    
    #randomises through randomise function below
    randomSwitch = False
    while randomSwitch == False:
        randomise_timetable(timetable)
        randomSwitch = room_validity(timetable)

    #for i in range(0, len(timetable)):
        #print(timetable[i])
    #print(timetableJoined)
    return timetable

In [67]:
def randomise_timetable(timetableI):
    #Create randomised timetable
    #Add all to single list
    #Shuffle list
    #Put back into timetable
    
    timetableJoined = []
    counter = 0
    for list in timetableI:
        for i in list:
            timetableJoined.append(i)
    np.random.shuffle(timetableJoined)
    for i in range(0,len(timetableI)):
        for j in range(0,3):
            timetableI[i][j] = timetableJoined[counter]
            counter+=1
    
    return timetableI

In [68]:
def room_validity(timetable):
    #Check to see if timetable is valid ie:
    #lectures take place in lecture theatre & labs are in labs
    #returns true if valid, else false if not
    """
    allSplits = []
    f = open("modules.txt", 'r')
    for line in f:
        allSplits.append(line.split("|"))
    f.close()
    """
    counter = -1
    for i in range(0,len(timetable)):
        current = timetable[i][0]
        if current=="empty":
            continue
        for j in range(0,len(timetable)):
            if i!=j:
                if current == timetable[j][0]:
                    if counter < 0:
                        counter = j
                    else:
                        return False
    
    #Checks that all lectures are in lecutre theatre (and therefore labs are in labs)
    allClasses = 0;
    for l in range(0,len(allSplits)):
        tree = allSplits[l][0]
        for m in range(0,len(timetable)):
            if allSplits[l][0] == timetable[m][0]:
                allClasses+=1
                break
    if allClasses != 17:
        return False
    return True             

In [69]:
def fitness_function(timetableV):
    """
    Generate a fitness calculator Spyx
    General setup directly below.
    Could probably be more efficient if
    unnecessary repetition was removed.
    """
    #timetableV = generate_timetable()
    #for i in range(0, len(timetableV)):
        #print(timetableV[i])
    
    """
    CONSTRAINTS:
    1) ONLY 1 LECTURE AND TWO LABS MAY RUN CONCURRENTLY
    2) LABS FOR A MODULE MAY ONLY OCCUR AFTER ITS LECTURE! PRECEDENCE!
    3) SESSIONS CAN'T INTERFERE! CONCURRENCE!
    """
    #Precedence constraints
    precConstraints = 0
    #module loop
    for i in range(0, len(timetableV)):
        module = timetableV[i][0]
        if module == "empty":
            continue
        #all of the timetable lab loop
        for j in range(0, len(timetableV)):
            if j > i:
                break
            for l in range(1, len(timetableV[j])):
                if module == timetableV[j][l]:
                    precConstraints+=1
            
    
    #Concurrency constraints
    #Functional and clean, aside from try catch section which only caters to list length 3.
    concConstraints = 0
    #Cycle through each session
    for i in range(0,len(timetableV)):
        #Cycle through each room
        for j in range(0,len(timetableV[i])):
            if timetableV[i][j] == "empty":
                continue
            constraintModules = []
            pos1 = modSplits.index(timetableV[i][j])
            clashes1 = modConstraintSplits[pos1]
            #2nd cycle through each room, linear comparison of modules and clashes
            for k in range(0,len(timetableV[i])):
                if k == j or timetableV[i][k] == "empty":
                    continue
                else:
                    pos2 = modSplits.index(timetableV[i][k])
                    clashes2 = modConstraintSplits[pos2]
                    if timetableV[i][j] in clashes2 or timetableV[i][k] in clashes1 or timetableV[i][j] == timetableV[i][k]:
                        concConstraints+=1

 
    """
    #for each column
    for h in range(0,len(timetableV[0])):
        #for each module
        for i in range(0, len(allSplits)):
            moduleP = allSplits[i][h]
            if moduleP == "empty":
                continue
            mpClashes = allSplits[i][3].split(",")
            mpClashes.append(moduleP)
            #for each row
            for j in range(0, len(timetableV)):
                if timetableV[j][h] == moduleP:
                    #for each item in row
                    for p in range(0,len(timetableV[0])):
                        if p != h:
                            if timetableV[j][p] in mpClashes:
                                concConstraints+=1
    """                 
        
    return precConstraints + concConstraints
    #print("precConstraints: ", precConstraints) 
    #print("concConstraints: ", concConstraints)

In [ ]:
#Optimisation Algorithm Implementation

#Mutate module
x, y = hillclimber(5000, fitness_function, mutation_function, comparison_lower)
print(f"Best Solution:")
for i in range(0, len(x)):
    print(x[i])
print(f"fitness: {y}")

#Randomise
a, b = hillclimber(5000, fitness_function, mutation_function_2, comparison_lower)
print(f"Best Solution:")
for i in range(0, len(a)):
    print(a[i])
print(f"fitness: {b}")

Progress: 0 / 5000
New best! Fitness = 65
New best! Fitness = 64
New best! Fitness = 63
New best! Fitness = 62
New best! Fitness = 61
Progress: 100 / 5000
New best! Fitness = 58
New best! Fitness = 54
New best! Fitness = 53
New best! Fitness = 52
Progress: 200 / 5000
New best! Fitness = 50
Progress: 300 / 5000
Progress: 400 / 5000
Progress: 500 / 5000
Progress: 600 / 5000
Progress: 700 / 5000
Progress: 800 / 5000
New best! Fitness = 49
Progress: 900 / 5000
Progress: 1000 / 5000
New best! Fitness = 47
New best! Fitness = 46
Progress: 1100 / 5000
Progress: 1200 / 5000
Progress: 1300 / 5000
Progress: 1400 / 5000
Progress: 1500 / 5000
Progress: 1600 / 5000
Progress: 1700 / 5000
Progress: 1800 / 5000
Progress: 1900 / 5000
Progress: 2000 / 5000
Progress: 2100 / 5000
Progress: 2200 / 5000
Progress: 2300 / 5000
Progress: 2400 / 5000
Progress: 2500 / 5000
Progress: 2600 / 5000
Progress: 2700 / 5000
Progress: 2800 / 5000
Progress: 2900 / 5000
Progress: 3000 / 5000
Progress: 3100 / 5000
Progress:

In [70]:
def mutation_function(timetable):
    #This function swaps two modules at random.
    moduleIndex = []
    tim = len(timetable)-1
    tim2 = len(timetable[0])-1
    moduleIndex.append(random.randint(0,tim))
    moduleIndex.append(random.randint(0,tim2)) 
    whileToggle = False
    x = random.randint(0,tim)
    y = random.randint(0,tim2) 
    while whileToggle == False:
        if x == moduleIndex[0] and y == moduleIndex[1]:
            x = random.randint(0,tim)
            y = random.randint(0,tim2)
        else:
            whileToggle = True
    moduleIndex.append(x)
    moduleIndex.append(y)
    mod1 = timetable[moduleIndex[0]][moduleIndex[1]]
    mod2 = timetable[moduleIndex[2]][moduleIndex[3]]
    timetable[moduleIndex[0]][moduleIndex[1]] = mod2
    timetable[moduleIndex[2]][moduleIndex[3]] = mod1
    return timetable
    

In [71]:
def mutation_function_2(timetable):
    return generate_timetable()

In [77]:
#Objective Function for current coursework problem.
def comparison_lower(a, b):
    return b <= a

In [80]:
def hillclimber(itInt, problem, mutation, comparison):
    #itInt = no. of iterations
    #problem = fitness calculator
    #mutation = function by which to change decision variable
    #decision variable is the timetable, generated within the function!
    #comparison = objective function for current optimisation goal
    parent = generate_timetable()
    #for i in range(0,len(parent)):
        #print(parent[i])
    parentFitness = problem(parent)
    
    #Loop for itint no. of times
    for iter in range(itInt):
        if iter % 100 == 0:
            print(f"Progress: {iter} / {itInt}")
        #Mutate
        child = mutation(parent)
        
        #Evaluate
        childFitness = problem(child)
        
        #Print parent soln
        if comparison(parentFitness, childFitness):
            if parentFitness > childFitness:
                print(f"New best! Fitness = {childFitness}")
            parent = child
            parentFitness = childFitness
    
    return parent, parentFitness